In [1]:
# Dependencies
import pandas as pd
import math

## Data Retrieval

In [2]:
# Read HPD Crime Report data and save as "crime_info_pd"
crime_info_pd =pd.read_csv("../data/cleandata/HPD_Crime_Report.csv")

# Preview "crime_info_pd"
crime_info_pd.head()

,Offense,Crime Type,Lat,Lng
0,Theft,Non-Violet,29.828978,-95.380827
1,Robbery,Violet,29.703565,-95.350084
2,Burglary,Non-Violet,29.767965,-95.318870
3,Burglary,Non-Violet,29.560540,-95.125612
4,Robbery,Violet,29.921638,-95.410273


In [3]:
# Read Houston Harris County zipcodes data and save as "harris_cty_zipcode_pd"
harris_cty_zipcode_pd = pd.read_csv("../data/cleandata/Harris_County_zipcode_lat_lngs.csv")

# Preview "harris_cty_zipcode_pd"
harris_cty_zipcode_pd.head()

,ZIP,LAT,LNG
0,77002,29.756845,-95.365652
1,77004,29.724893,-95.363752
2,77003,29.749778,-95.345885
3,77006,29.740970,-95.391301
4,77005,29.718435,-95.423555


## Crime Counts Calculator

In [4]:
# Copy "harris_cty_zipcode_pd" and save as "crime_ct_by_zipcode"
crime_ct_by_zipcode = harris_cty_zipcode_pd

# Preview "crime_ct_by_zipcode"
crime_ct_by_zipcode.head()

,ZIP,LAT,LNG
0,77002,29.756845,-95.365652
1,77004,29.724893,-95.363752
2,77003,29.749778,-95.345885
3,77006,29.740970,-95.391301
4,77005,29.718435,-95.423555


In [5]:
# Set a radius for the search of nearby crimes
searching_radius = input("Please provide a searching radius (in miles): ")

Please provide a searching radius (in miles): 2


In [6]:
# use "violent_crime_sum_radius" to store the total number of violent crime within given miles
violent_crime_ct = []
# use "non_violent_crime_sum_radius" to store the total number of non-violent crime within given miles
non_violent_crime_ct = []


In [7]:
# Use "radius_0" to store the radius of the earth (3958.8 mi)
radius_0 = 3958.8

In [8]:
# Calculate the mile coverage on longitude line per degree of latitude change on earth (2 * math.pi * radius_0 / 360)
lat_multiplier = 69.1

In [9]:
# Loop through "crime_info_pd"
for i in range(len(harris_cty_zipcode_pd)):
    
    # Calculate the radius of the latitude line on earth (crime #i) and assign the value in "radius_lat"
    # Note that math.cos() takes radians rather than degree
    radius_lat = radius_0 * math.cos(harris_cty_zipcode_pd.LAT[i] * math.pi / 180)
    
    # Calculate the miles coverage on latitude line (crime #i) per degree of longitude change on earth
    lng_multiplier = 2 * math.pi * radius_lat / 360
    
    # Set counters for violent and non-violent crimes during each iteration
    violent_counter = 0
    non_violent_counter = 0
    
    # Loop through "crime_info_pd" for crimes within given searching radius
    for j in range(len(crime_info_pd)):
        
        # Calculate loc #i' and #j's distance along the lantitude and longitude lines of loc #i 
        dist_lat = abs(crime_info_pd.Lat[j] - harris_cty_zipcode_pd.LAT[i]) * lat_multiplier
        dist_lng = abs(crime_info_pd.Lng[j] - harris_cty_zipcode_pd.LNG[i]) * lng_multiplier
        
        # Check if crime #j is within the searching radius from loc #i, and if true ...
        if (dist_lat ** 2 + dist_lng ** 2) <= float(searching_radius) ** 2:
            
            # Add 1 to violent crime counter if crime #j is a violent type of crime
            if crime_info_pd["Crime Type"][j] == "Violent":              
                violent_counter += 1                
            # Add 1 to non-violent crime counter if crime #j is a non-violent type of crime
            if crime_info_pd["Crime Type"][j] == "Non-Violent":
                non_violent_counter += 1  
        
    # Append violent crime counts to "violent_crime_ct" 
    violent_crime_ct.append(violent_counter)
    # Append non-violent crime counts to "non_violent_crime_ct"     
    non_violent_crime_ct.append(non_violent_counter)

# Add new column of "Violent Crime Ct" to "crime_ct_by_zipcode"
crime_ct_by_zipcode["Violent Crime Ct"] = violent_crime_ct
# Add new column of "Sum Non-Violent Crime within Given Radius" to "crime_ct_by_zipcode"
crime_ct_by_zipcode["Non-Violent Crime Ct"] = non_violent_crime_ct

# Preview "crime_ct_by_zipcode"
crime_ct_by_zipcode.head(10)

,ZIP,LAT,LNG,Violet Crime Ct,Non-Violet Crime Ct
0,77002,29.756845,-95.365652,83,506
1,77004,29.724893,-95.363752,78,366
2,77003,29.749778,-95.345885,84,351
3,77006,29.740970,-95.391301,76,550
4,77005,29.718435,-95.423555,22,271
5,77008,29.798249,-95.416933,18,222
6,77007,29.771545,-95.411083,14,278
7,77010,29.753624,-95.359810,93,484
8,77009,29.795344,-95.367590,38,152
9,77012,29.718525,-95.274137,24,74


In [11]:
# Write "crime_ct_by_zipcode" to csv file
crime_ct_by_zipcode.to_csv("../data/rawdata/Harris_County_Crime_Ct_by_zipcode_2miles.csv", index=False, header=True)